In [36]:
import heapq
import random

In [37]:
# --- Cliente ---
class Customer:
    def __init__(self, id, arrival_time):
        self.id = id
        self.arrival_time = arrival_time
        self.departure_time = None

In [38]:
# --- Evento ---
class Event:
    def __init__(self, time, type, customer_id):
        self.time = time
        self.type = type  # 'arrival', 'departure_1', 'departure_2'
        self.customer_id = customer_id

    def __lt__(self, other):
        return self.time < other.time  # Sobrecarga de operador '<' para indicar a python como comparar dos eventos usando el tiempo

In [39]:
# --- Simulador ---
class Simulator:
    def __init__(self, max_customers=1000):
        self.t = 0  # tiempo actual
        self.event_list = []  # lista de eventos (heapq)
        self.customers = {}  # id -> Customer
        self.queue = []  # cola de espera (ids de clientes)

        self.server1 = None  # cliente atendido por servidor 1
        self.server2 = None  # cliente atendido por servidor 2

        self.next_customer_id = 1
        self.max_customers = max_customers
        self.C1 = 0  # servicios por servidor 1
        self.C2 = 0  # servicios por servidor 2

    def generate_interarrival_time(self):
        return random.expovariate(1.0)  # media 1

    def generate_service_time_1(self):
        return random.expovariate(1.5)

    def generate_service_time_2(self):
        return random.expovariate(1.0)

    def schedule_event(self, event):
        heapq.heappush(self.event_list, event)

    def start(self):
        # Generamos primer evento de llegada
        first_arrival_time = self.t + self.generate_interarrival_time()
        self.schedule_event(Event(first_arrival_time, 'arrival', self.next_customer_id))

        # Bucle de simulación
        while self.event_list and self.next_customer_id <= self.max_customers:
            event = heapq.heappop(self.event_list)
            self.t = event.time
            self.handle_event(event)

        self.print_stats()

    def handle_event(self, event):
        if event.type == 'arrival':
            self.handle_arrival(event)
        elif event.type == 'departure_1':
            self.handle_departure(event, server=1)
        elif event.type == 'departure_2':
            self.handle_departure(event, server=2)

    def handle_arrival(self, event):
        cid = event.customer_id
        customer = Customer(cid, self.t)
        self.customers[cid] = customer

        # Programamos próxima llegada
        self.next_customer_id += 1
        if self.next_customer_id <= self.max_customers:
            next_arrival = self.t + self.generate_interarrival_time()
            self.schedule_event(Event(next_arrival, 'arrival', self.next_customer_id))

        if self.server1 is None:
            self.server1 = cid
            service_time = self.generate_service_time_1()
            self.schedule_event(Event(self.t + service_time, 'departure_1', cid))
        elif self.server2 is None:
            self.server2 = cid
            service_time = self.generate_service_time_2()
            self.schedule_event(Event(self.t + service_time, 'departure_2', cid))
        else:
            self.queue.append(cid)

    def handle_departure(self, event, server):
        cid = event.customer_id
        self.customers[cid].departure_time = self.t

        if server == 1:
            self.C1 += 1
            self.server1 = None
        else:
            self.C2 += 1
            self.server2 = None

        if self.queue:
            next_cid = self.queue.pop(0)
            if server == 1:
                self.server1 = next_cid
                service_time = self.generate_service_time_1()
                self.schedule_event(Event(self.t + service_time, 'departure_1', next_cid))
            else:
                self.server2 = next_cid
                service_time = self.generate_service_time_2()
                self.schedule_event(Event(self.t + service_time, 'departure_2', next_cid))

    def print_stats(self):
        total_time = 0
        for c in self.customers.values():
            if c.departure_time:
                time_in_system = c.departure_time - c.arrival_time
                total_time += time_in_system
                print(f"Cliente {c.id} - Tiempo en sistema: {time_in_system:.2f}")
        print(f"\nTotal clientes: {len(self.customers)}")
        print(f"Tiempo promedio en el sistema: {total_time / len(self.customers):.2f}")
        print(f"Servicios servidor 1: {self.C1}")
        print(f"Servicios servidor 2: {self.C2}")

In [40]:
# Ejecutar la simulación
sim = Simulator(max_customers=50)
sim.start()

Cliente 1 - Tiempo en sistema: 1.01
Cliente 2 - Tiempo en sistema: 1.05
Cliente 3 - Tiempo en sistema: 1.71
Cliente 4 - Tiempo en sistema: 0.28
Cliente 5 - Tiempo en sistema: 0.14
Cliente 6 - Tiempo en sistema: 0.44
Cliente 7 - Tiempo en sistema: 1.43
Cliente 8 - Tiempo en sistema: 0.33
Cliente 9 - Tiempo en sistema: 1.41
Cliente 10 - Tiempo en sistema: 0.63
Cliente 11 - Tiempo en sistema: 0.73
Cliente 12 - Tiempo en sistema: 1.61
Cliente 13 - Tiempo en sistema: 2.25
Cliente 14 - Tiempo en sistema: 0.86
Cliente 15 - Tiempo en sistema: 0.87
Cliente 16 - Tiempo en sistema: 0.08
Cliente 17 - Tiempo en sistema: 0.26
Cliente 18 - Tiempo en sistema: 0.78
Cliente 19 - Tiempo en sistema: 0.87
Cliente 20 - Tiempo en sistema: 0.33
Cliente 21 - Tiempo en sistema: 0.26
Cliente 22 - Tiempo en sistema: 1.40
Cliente 23 - Tiempo en sistema: 0.88
Cliente 24 - Tiempo en sistema: 1.56
Cliente 25 - Tiempo en sistema: 1.21
Cliente 26 - Tiempo en sistema: 1.18
Cliente 27 - Tiempo en sistema: 0.89
Cliente 28